In [ ]:
!pip install openai
!pip install dotenv

In [7]:
import dotenv

%load_ext dotenv
%dotenv


In [8]:
import openai
import os

openai.api_type = "azure"
openai.api_base = "https://openaimg01.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
messages = []

def chat(prompt):
  messages.append({"content": prompt, "role": "user"})
  response = openai.ChatCompletion.create(
      engine="SebastianGPT35Turbo",
      messages = messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)
  message = response.choices[0].message
  messages.append(message)
  return message.content.strip()

In [10]:
user_input = ""
while user_input != "bye":
    user_input = input("You: ")
    print(f"You: {user_input}\n")
    ai_response = chat(user_input)
    print(f"AI: {ai_response}\n")

You: hello

AI: Hello! How can I assist you today?

You: wer hat am ESC23 gewonnen?

AI: Ich bin mir nicht sicher, da der ESC 2023 noch nicht stattgefunden hat. Der ESC 2022 soll im Mai in Italien stattfinden.

You: byew

AI: Goodbye! Have a great day!

You: bye

AI: Goodbye! Take care!

